<a href="https://colab.research.google.com/github/Maicon-F/AI/blob/101-TransFer-Learning/notebooks/transfer-learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

import os

#if using Theano with GPU
#os.environ["KERAS_BACKEND"] = "tensorflow"

import random
import numpy as np
import keras

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model

Using TensorFlow backend.


[link text](https://)

# Download dataset

# Prepare The Database






In [ ]:
from keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(rescale=1./255, # Scale pixel values
                                   rotation_range=20, # Data augmentation
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'training',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    'validation',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

num_classes = train_generator.num_classes

Using TensorFlow backend.


# Build the Transfer Learning Model

In [ ]:
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout

# Load VGG16 without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add your new classification layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the new model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile and Train the Model

In [ ]:
from keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator)

# Evaluate and Predict

In [ ]:
import numpy as np
from keras.preprocessing import image

# Create a test generator (do not shuffle)
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    'testing',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False)

# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy:.2f}')

# --- Predict a single image ---
def get_single_image_vector(path):
    img = image.load_img(path, target_size=IMG_SIZE)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0) # Add batch dimension
    return x

# Assuming 'path/to/your/new_image.jpg' is your image
img_vector = get_single_image_vector('path/to/your/new_image.jpg')
prediction = model.predict(img_vector)
predicted_class_index = np.argmax(prediction)

# Get the class names from the generator
class_labels = list(train_generator.class_indices.keys())
predicted_class = class_labels[predicted_class_index]
probability = np.max(prediction)

print(f"Prediction: {predicted_class} with probability {probability:.2f}")